In [1]:
import torch
import numpy as np
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from typing import List, Dict
from pathlib import Path

In [ ]:
# ===============================
# Generic HF CLS Embedder
# ===============================
class HFCLSEmbedder:
    def __init__(self, model_path: str, device: str = 'cpu'):
        model_path = str(Path(model_path).resolve())
        self.model_path = model_path
        self.device = device
        
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_path,
            local_files_only=True
        )

        self.model = AutoModel.from_pretrained(
            model_path,
            local_files_only=True
        ).to(device)

        self.model.eval()

    def encode(self, texts: List[str]) -> np.ndarray:
        with torch.no_grad():
            inputs = self.tokenizer(
                texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors="pt"
            )
            outputs = self.model(**inputs)

            # CLS token embedding
            cls_embeddings = outputs.last_hidden_state[:, 0]

            # Normalize for cosine similarity
            cls_embeddings = F.normalize(cls_embeddings, dim=1)

            return cls_embeddings.cpu().numpy()


# ===============================
# Cosine similarity
# ===============================
def cosine_sim(a: np.ndarray, b: np.ndarray) -> float:
    return float(np.dot(a, b))

In [4]:
# ===============================
# Load all models ONCE
# ===============================
def load_embedders(models: Dict[str, str]) -> Dict[str, HFCLSEmbedder]:
    embedders = {}

    for name, model_path in models.items():
        print(f"\n🔍 Loading {name} ...")
        embedders[name] = HFCLSEmbedder(model_path)

    return embedders

In [5]:
# ===============================
# Benchmark runner
# ===============================
def benchmark_similarity(
    embedders: Dict[str, HFCLSEmbedder],
    text_pairs: List[List[str]]
):
    results = {}

    for name, embedder in embedders.items():
        print(f"\n📊 Benchmarking {name} ...")

        scores = []
        for t1, t2 in text_pairs:
            emb = embedder.encode([t1, t2])
            sim = cosine_sim(emb[0], emb[1])
            scores.append(sim)

        results[name] = {
            "mean_similarity": float(np.mean(scores)),
            "min": float(np.min(scores)),
            "max": float(np.max(scores)),
            "scores": scores,
        }

    return results


# ===============================
# MAIN
# ===============================
if __name__ == "__main__":

    TEXT_PAIRS = [
        (
            "Heart rate variability reflects autonomic nervous system function.",
            "HRV is an indicator of autonomic nervous system activity."
        ),
        (
            "Poor sleep quality increases cardiovascular risk.",
            "Lack of sleep is associated with higher risk of heart disease."
        ),
        (
            "Vitamin D deficiency affects bone health.",
            "Vitamin D is essential for maintaining bone density."
        ),
        (
            "Insulin regulates blood glucose levels.",
            "Blood sugar is controlled by insulin."
        ),
        (
            "Heart rate variability reflects autonomic nervous system function.",
            "Vitamin D is essential for bone health."
        )
    ]

    MODELS = {
        "PubMedBERT": "./models/pubmedbert",
        "ClinicalBERT": "./models/clinicalbert",
        "BioELECTRA": "./models/bioelectra",
        "SapBERT": "./models/sapbert",
        "BioBERT": "./models/biobert-stsb",
        "MiniLM-L6": "./models/all-MiniLM-L6-v2",
    }

    embedders = load_embedders(MODELS)
    results = benchmark_similarity(embedders, TEXT_PAIRS)

    print("\n================= RESULTS =================")
    for model, stats in results.items():
        print(f"\n🧠 {model}")
        print(f"  Mean similarity: {stats['mean_similarity']:.4f}")
        print(f"  Min similarity : {stats['min']:.4f}")
        print(f"  Max similarity : {stats['max']:.4f}")


🔍 Loading PubMedBERT ...

🔍 Loading ClinicalBERT ...

🔍 Loading BioELECTRA ...

🔍 Loading SapBERT ...

🔍 Loading BioBERT ...

🔍 Loading MiniLM-L6 ...

📊 Benchmarking PubMedBERT ...

📊 Benchmarking ClinicalBERT ...

📊 Benchmarking BioELECTRA ...

📊 Benchmarking SapBERT ...

📊 Benchmarking BioBERT ...

📊 Benchmarking MiniLM-L6 ...

================= RESULTS =================

🧠 PubMedBERT
  Mean similarity: 0.9567
  Min similarity : 0.9120
  Max similarity : 0.9852

🧠 ClinicalBERT
  Mean similarity: 0.9142
  Min similarity : 0.8315
  Max similarity : 0.9467

🧠 BioELECTRA
  Mean similarity: 0.9478
  Min similarity : 0.9202
  Max similarity : 0.9706

🧠 SapBERT
  Mean similarity: 0.7310
  Min similarity : 0.2902
  Max similarity : 0.9113

🧠 BioBERT
  Mean similarity: 0.7216
  Min similarity : 0.2975
  Max similarity : 0.8874

🧠 MiniLM-L6
  Mean similarity: 0.8129
  Min similarity : 0.4990
  Max similarity : 0.9578
